In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from scipy.constants import Boltzmann, electron_volt
from pymatgen import units

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

In [9]:
#read in energies
#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies1 = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies2 = pd.read_csv('data/energy.out.1x1x4.csv', header=None)
  

#combine all energies
qe_energies = pd.concat([qe_energies1, qe_energies2])
#qe_energies_all = qe_energies_all.reset_index(drop=True).drop_duplicates(["pattern"], keep="first") 
qe_energies.columns = ["pattern", "num_sites", "total_energy" ]
qe_energies.head()


,pattern,num_sites,total_energy
0,nm,24,-14691.879405
1,c-1,24,-14693.951620
2,c-2,24,-14693.957454
3,c-3,24,-14693.970500
4,c-4,24,-14693.947722


In [10]:
exchange_model = pd.read_csv("data/exchange_model.csv")
exchange_model = exchange_model.drop(['Unnamed: 0'], axis=1)
exchange_model.tail()

,pattern,J1,J2,J3,J4,J5,J6,J7,J8
32,z-5,0.000000,0.000000,1.000000,-0.666667,0.000000,0.333333,1.000000,0.666667
33,z-6,0.333333,0.000000,1.000000,0.000000,0.666667,0.000000,1.000000,0.666667
34,z-7,0.000000,0.000000,0.333333,-0.666667,0.000000,-0.333333,0.333333,0.666667
35,z-8,0.000000,0.000000,0.666667,-1.333333,0.000000,0.000000,0.666667,0.666667
36,z-9,0.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.666667,0.666667


In [26]:
model_matrix = pd.merge(qe_energies, exchange_model,  on =["pattern"])
model_matrix = model_matrix.drop_duplicates(["pattern"]).reset_index(drop=True)


    
nm_energy = np.float64(model_matrix.query("pattern == 'nm'").loc[:, "total_energy"])
model_matrix["total_energy"] = ( (model_matrix["total_energy"] \
                                 - np.float64(model_matrix.query("pattern == 'nm'").loc[:, "total_energy"]))\
                                 / model_matrix["num_sites"] )\
                                 * 1000 * units.Ry_to_eV
    

model_matrix = model_matrix[model_matrix.pattern != "nm"].drop("num_sites", axis=1)

#model_matrix

#### Checking fitting results relative to the number of patterns included in the fit:

This doesn't affect the results of the fit in both cases.

In [12]:
smf_exchange_parameters= pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"])
for i in range(10, len(model_matrix)):
    smf_exchange_fit = smf.ols(data=model_matrix[:i], formula="total_energy ~ J1 + J2 + J3 + J4 +J5 + J6 + J7 + J8 ").fit()
    #smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
    smf_exchange_parameters["model_"+str(i)] = pd.DataFrame(smf_exchange_fit.params)
    
smf_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-685.741911,-685.741292,-685.829118,-686.452805,-686.198552,-686.470109,-684.464944,-685.024352,-684.617586,-684.564835,...,-687.402040,-691.005472,-694.036409,-694.600697,-695.186217,-697.628322,-696.546017,-696.996343,-696.321686,-696.321412
J1,52.402370,52.480682,52.506298,52.262246,52.104302,52.108939,53.137904,52.986972,52.913740,52.937569,...,53.207356,52.439353,51.229692,52.550128,53.031814,51.935735,52.605897,52.948350,53.397828,53.420821
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-136.182835,-136.264240,-135.850727,-135.240599,-134.845737,-134.483957,-136.983246,-136.988038,-137.573918,-138.162281,...,-140.896330,-137.851018,-135.198991,-135.462379,-135.596672,-133.190818,-134.925537,-134.985770,-136.428263,-136.738104
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033
J5,104.804740,104.961364,105.012595,104.524493,104.208603,104.217877,106.275808,105.973943,105.827481,105.875139,...,106.414712,104.878707,102.459384,105.100256,106.063627,103.871471,105.211794,105.896700,106.795656,106.841643
J6,268.576069,268.415939,268.862387,270.132357,270.515662,271.053844,266.188812,266.707892,265.924065,265.276706,...,264.164341,270.379943,276.262256,275.054624,274.828992,279.958995,276.832577,276.730109,274.388366,274.055349
J7,-136.182835,-136.264240,-135.850727,-135.240599,-134.845737,-134.483957,-136.983246,-136.988038,-137.573918,-138.162281,...,-140.896330,-137.851018,-135.198991,-135.462379,-135.596672,-133.190818,-134.925537,-134.985770,-136.428263,-136.738104
J8,-457.161274,-457.160862,-457.219412,-457.635203,-457.465701,-457.646740,-456.309963,-456.682901,-456.411724,-456.376557,...,-458.268027,-460.670314,-462.690939,-463.067131,-463.457478,-465.085548,-464.364012,-464.664229,-464.214457,-464.214275


In [13]:
lm = LinearRegression()
lm_exchange_parameters = pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"])

for i in range(10, len(model_matrix)):
    model_matrix_i = model_matrix[:i]
    lm_exchange_fit = lm.fit(X=model_matrix_i[["J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"]], 
                             y=model_matrix_i["total_energy"])
    parameters = [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3], lm_exchange_fit.coef_[4],lm_exchange_fit.coef_[5],
                lm_exchange_fit.coef_[6], lm_exchange_fit.coef_[7]]
    
    lm_exchange_parameters ["model_"+str(i)]= parameters
 
 
lm_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1.169795e+03,-1.169794e+03,-1169.943789,-1171.007725,-1170.574000,-1.171037e+03,-1167.616669,-1168.570954,-1.167877e+03,-1.167787e+03,...,-1.172627e+03,-1.178774e+03,-1.183944e+03,-1.184907e+03,-1.185906e+03,-1.190072e+03,6.618928e+14,-1188.993762,-1187.842876,-1.187842e+03
J1,-1.381309e+00,-1.302949e+00,-1.284221,-1.577189,-1.715193,-1.731854e+00,-0.545621,-0.740429,-7.817565e-01,-7.537903e-01,...,-7.065295e-01,-1.757154e+00,-3.204536e+00,-1.928358e+00,-1.492596e+00,-2.780212e+00,7.458685e+13,-1.718030,-1.215637,-1.192623e+00
J2,-3.980287e+00,-3.818817e+00,-4.455553,-4.324488,-5.665093,-5.800280e+00,-0.917224,-0.917224,-9.172235e-01,-9.172235e-01,...,2.487116e+00,-5.505439e+00,-1.018522e+01,-1.236361e+01,-1.239723e+01,-1.678038e+01,-1.368359e+01,-13.822597,-9.383758,-9.003992e+00
J3,-1.723637e+00,-1.805163e+00,-1.374430,-0.642010,-0.297002,1.180254e-01,-2.774434,-2.669537,-3.335176e+00,-3.933882e+00,...,-6.111616e+00,-2.359749e+00,8.865789e-01,7.338358e-01,7.143506e-01,3.599049e+00,1.324485e+14,1.680180,0.105401,-2.044938e-01
J4,-5.330468e+00,-5.290101e+00,-5.449285,-6.104608,-8.283090,-8.012716e+00,-7.911155,-6.122126,-6.586820e+00,-6.954305e+00,...,1.568727e+00,1.301827e+00,2.305443e+00,5.352866e-01,1.663675e-01,1.039477e+00,2.121094e+00,1.815324,1.857108,3.178033e+00
J5,-2.762619e+00,-2.605898e+00,-2.568443,-3.154378,-3.430385,-3.463708e+00,-1.091242,-1.480857,-1.563513e+00,-1.507581e+00,...,-1.413059e+00,-3.514308e+00,-6.409072e+00,-3.856716e+00,-2.985191e+00,-5.560423e+00,-2.204903e+14,-3.436060,-2.431275,-2.385245e+00
J6,-3.423271e-01,-5.022144e-01,-0.090208,0.935179,1.418191,1.849880e+00,-2.228813,-1.929109,-2.553420e+00,-3.180092e+00,...,-5.405087e+00,-6.025951e-01,4.091115e+00,2.662194e+00,2.206946e+00,6.379261e+00,-3.663938e+14,3.398210,1.321038,9.881289e-01
J7,-1.723637e+00,-1.805163e+00,-1.374430,-0.642010,-0.297002,1.180254e-01,-2.774434,-2.669537,-3.335176e+00,-3.933882e+00,...,-6.111616e+00,-2.359749e+00,8.865789e-01,7.338358e-01,7.143506e-01,3.599049e+00,2.339454e+14,1.680180,0.105401,-2.044938e-01
J8,1.123455e-30,-1.449869e-30,0.000000,0.000000,0.000000,-5.085121e-30,0.000000,0.000000,-8.090963e-31,1.009593e-30,...,-2.788035e-31,1.406401e-31,-1.709747e-30,-2.584228e-29,9.578489e-30,-4.356427e-29,-1.359233e+15,0.000000,0.000000,1.609713e-30


#### Checking relative to the number of parameters:

The divergence in calculated values occurs when 6 parameters are included in the fit. The linear model results are consistent with more parameters, the statsmodel results change significantly.

- Reduce to 4 J parameters

In [17]:
smf_exchange_parameters= pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4"])
for i in range(10, len(model_matrix)):
    smf_exchange_fit = smf.ols(data=model_matrix[:i], formula="total_energy ~ J1 + J2 + J3 + J4").fit()
    #smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
    smf_exchange_parameters["model_"+str(i)] = pd.DataFrame(smf_exchange_fit.params)
    
smf_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1169.566806,-1169.459160,-1169.883651,-1171.631178,-1171.519461,-1172.270499,-1166.130794,-1167.284881,-1166.174778,-1165.667010,...,-1169.023618,-1178.372310,-1186.671872,-1186.681867,-1187.377197,-1194.324684,-1190.677076,-1191.259235,-1188.723568,-1188.501162
J1,-6.564220,-6.012530,-6.330898,-8.821125,-9.994154,-10.509151,-0.499292,-1.773035,-1.355363,-0.588860,...,1.872439,-8.183175,-20.113796,-12.303984,-9.669924,-20.280319,-13.803091,-11.988359,-7.399226,-6.951243
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-3.789600,-4.112541,-2.839068,-0.348841,0.824188,2.085930,-7.777680,-7.268183,-9.223772,-11.047856,...,-17.628319,-5.322093,5.864273,4.129866,3.635647,13.577359,6.981502,6.758570,1.531840,0.579141
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033


In [18]:
lm = LinearRegression()
lm_exchange_parameters = pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4"])

for i in range(10, len(model_matrix)):
    model_matrix_i = model_matrix[:i]
    lm_exchange_fit = lm.fit(X=model_matrix_i[["J1", "J2", "J3", "J4"]], 
                             y=model_matrix_i["total_energy"])
    parameters = [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3]]
    
    lm_exchange_parameters ["model_"+str(i)]= parameters
 
 
lm_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1169.566806,-1169.459160,-1169.883651,-1171.631178,-1171.519461,-1172.270499,-1166.130794,-1167.284881,-1166.174778,-1165.667010,...,-1169.023618,-1178.372310,-1186.671872,-1186.681867,-1187.377197,-1194.324684,-1190.677076,-1191.259235,-1188.723568,-1188.501162
J1,-6.564220,-6.012530,-6.330898,-8.821125,-9.994154,-10.509151,-0.499292,-1.773035,-1.355363,-0.588860,...,1.872439,-8.183175,-20.113796,-12.303984,-9.669924,-20.280319,-13.803091,-11.988359,-7.399226,-6.951243
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-3.789600,-4.112541,-2.839068,-0.348841,0.824188,2.085930,-7.777680,-7.268183,-9.223772,-11.047856,...,-17.628319,-5.322093,5.864273,4.129866,3.635647,13.577359,6.981502,6.758570,1.531840,0.579141
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033


- 5 Parameters:

In [19]:
smf_exchange_parameters= pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5"])
for i in range(10, len(model_matrix)):
    smf_exchange_fit = smf.ols(data=model_matrix[:i], formula="total_energy ~ J1 + J2 + J3 + J4 + J5").fit()
    #smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
    smf_exchange_parameters["model_"+str(i)] = pd.DataFrame(smf_exchange_fit.params)
    
smf_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1169.566806,-1169.459160,-1169.883651,-1171.631178,-1171.519461,-1172.270499,-1166.130794,-1167.284881,-1166.174778,-1165.667010,...,-1169.023618,-1178.372310,-1186.671872,-1186.681867,-1187.377197,-1194.324684,-1190.677076,-1191.259235,-1188.723568,-1188.501162
J1,-1.312844,-1.202506,-1.266180,-1.764225,-1.998831,-2.101830,-0.099858,-0.354607,-0.271073,-0.117772,...,0.374488,-1.636635,-4.022759,-2.460797,-1.933985,-4.056064,-2.760618,-2.397672,-1.479845,-1.390249
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-3.789600,-4.112541,-2.839068,-0.348841,0.824188,2.085930,-7.777680,-7.268183,-9.223772,-11.047856,...,-17.628319,-5.322093,5.864273,4.129866,3.635647,13.577359,6.981502,6.758570,1.531840,0.579141
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033
J5,-2.625688,-2.405012,-2.532359,-3.528450,-3.997662,-4.203660,-0.199717,-0.709214,-0.542145,-0.235544,...,0.748976,-3.273270,-8.045518,-4.921594,-3.867970,-8.112127,-5.521236,-4.795343,-2.959690,-2.780497


In [22]:
lm = LinearRegression()
lm_exchange_parameters = pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5"])

for i in range(10, len(model_matrix)):
    model_matrix_i = model_matrix[:i]
    lm_exchange_fit = lm.fit(X=model_matrix_i[["J1", "J2", "J3", "J4", "J5"]], 
                             y=model_matrix_i["total_energy"])
    parameters = [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3], lm_exchange_fit.coef_[4] ]
    
    lm_exchange_parameters ["model_"+str(i)]= parameters
 
 
lm_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1169.566806,-1169.459160,-1169.883651,-1171.631178,-1171.519461,-1172.270499,-1166.130794,-1167.284881,-1166.174778,-1165.667010,...,-1169.023618,-1178.372310,-1186.671872,-1186.681867,-1187.377197,-1194.324684,-1190.677076,-1191.259235,-1188.723568,-1188.501162
J1,-1.312844,-1.202506,-1.266180,-1.764225,-1.998831,-2.101830,-0.099858,-0.354607,-0.271073,-0.117772,...,0.374488,-1.636635,-4.022759,-2.460797,-1.933985,-4.056064,-2.760618,-2.397672,-1.479845,-1.390249
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-3.789600,-4.112541,-2.839068,-0.348841,0.824188,2.085930,-7.777680,-7.268183,-9.223772,-11.047856,...,-17.628319,-5.322093,5.864273,4.129866,3.635647,13.577359,6.981502,6.758570,1.531840,0.579141
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033
J5,-2.625688,-2.405012,-2.532359,-3.528450,-3.997662,-4.203660,-0.199717,-0.709214,-0.542145,-0.235544,...,0.748976,-3.273270,-8.045518,-4.921594,-3.867970,-8.112127,-5.521236,-4.795343,-2.959690,-2.780497


- 6 Parameters

In [23]:
smf_exchange_parameters= pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6"])
for i in range(10, len(model_matrix)):
    smf_exchange_fit = smf.ols(data=model_matrix[:i], formula="total_energy ~ J1 + J2 + J3 + J4 + J5 + J6").fit()
    #smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
    smf_exchange_parameters["model_"+str(i)] = pd.DataFrame(smf_exchange_fit.params)
    
smf_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-973.625349,-973.645024,-973.661076,-974.361556,-973.913749,-974.194509,-972.078010,-972.845467,-972.436000,-972.512072,...,-977.087583,-981.255650,-984.738692,-985.578025,-986.413896,-989.152446,-988.107285,-988.739303,-988.178846,-988.256582
J1,57.469593,57.541735,57.600593,57.416662,57.282883,57.320967,58.115977,57.977217,57.850561,57.828710,...,57.955298,57.498363,56.557195,57.870356,58.355006,57.495608,58.010319,58.358308,58.683571,58.683125
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-297.701786,-297.833744,-297.172930,-296.253275,-295.584381,-295.028054,-298.856855,-298.927305,-299.831939,-300.780264,...,-305.532371,-300.997082,-297.035497,-297.525898,-297.809305,-294.180998,-296.873184,-297.021329,-299.285243,-299.787728
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033
J5,114.939186,115.083469,115.201185,114.833323,114.565766,114.641934,116.231953,115.954435,115.701122,115.657419,...,115.910597,114.996725,113.114390,115.740712,116.710011,114.991216,116.020638,116.716616,117.367143,117.366250
J6,293.912186,293.721204,294.333862,295.904434,296.408569,297.113985,291.079175,291.659122,290.608167,289.732408,...,287.904053,295.674989,302.899770,301.655763,301.444953,307.758358,303.854687,303.779898,300.817083,300.366869


In [25]:
lm = LinearRegression()
lm_exchange_parameters = pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6"])

for i in range(10, len(model_matrix)):
    model_matrix_i = model_matrix[:i]
    lm_exchange_fit = lm.fit(X=model_matrix_i[["J1", "J2", "J3", "J4", "J5", "J6"]], 
                             y=model_matrix_i["total_energy"])
    parameters = [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3], lm_exchange_fit.coef_[4],lm_exchange_fit.coef_[5] ]
    
    lm_exchange_parameters ["model_"+str(i)]= parameters
 
 
lm_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17,model_18,model_19,...,model_25,model_26,model_27,model_28,model_29,model_30,model_31,model_32,model_33,model_34
Intercept,-1170.317339,-1170.340988,-1170.360283,-1171.202274,-1170.664001,-1171.001480,-1168.457406,-1169.379905,-1168.887717,-1168.979157,...,-1174.479014,-1179.489115,-1183.675801,-1184.684697,-1185.689430,-1188.981223,-1187.724919,-1188.484616,-1187.810937,-1187.904377
J1,-1.538004,-1.467055,-1.409169,-1.635554,-1.742193,-1.721125,-0.797842,-0.983114,-1.084954,-1.111416,...,-1.262131,-1.971677,-3.123938,-1.861646,-1.427655,-2.453025,-1.874971,-1.565286,-1.206056,-1.211213
J2,-3.980287,-3.818817,-4.455553,-4.324488,-5.665093,-5.800280,-0.917224,-0.917224,-0.917224,-0.917224,...,2.487116,-5.505439,-10.185219,-12.363606,-12.397231,-16.780377,-13.730136,-13.822597,-9.383758,-9.003992
J3,-2.663802,-2.789798,-2.124119,-0.992197,-0.459003,0.182403,-4.287761,-4.125649,-5.154363,-6.079636,...,-9.445225,-3.646885,1.370167,1.134110,1.103996,5.562167,2.553266,2.596642,0.162892,-0.316036
J4,-5.330468,-5.290101,-5.449285,-6.104608,-8.283090,-8.012716,-7.911155,-6.122126,-6.586820,-6.954305,...,1.568727,1.301827,2.305443,0.535287,0.166368,1.039477,2.130974,1.815324,1.857108,3.178033
J5,-3.076007,-2.934109,-2.818339,-3.271107,-3.484385,-3.442249,-1.595685,-1.966228,-2.169909,-2.222832,...,-2.524262,-3.943353,-6.247876,-3.723291,-2.855309,-4.906051,-3.749942,-3.130572,-2.412111,-2.422426
J6,-1.125798,-1.322743,-0.714949,0.643356,1.283190,1.903528,-3.489919,-3.142535,-4.069409,-4.968220,...,-8.183094,-1.675208,4.494105,2.995756,2.531651,8.015192,4.428237,4.161928,1.368948,0.895177
